In [13]:
import pandas as pd
from spatial_helper.display import generate_cust_map
import folium
import  geopandas


# Visualising Ambulance Demand and Deprivation
Our goal was to visualise the spatial distribution of ambulances around England, and to examine how this correlated to deprivation.

The core challenge was that our datasets do not cleanly overlay over standardised spatial units (eg, LSOA) - instead, we have to aggregate to average by Ambulance Trust, which makes the data far less useful.

We begin by ingesting ambulance trust geolocations, as well as fuel poverty by LSOA, and join these together by mean using a spatial join.

In [14]:

ambulance_trust = geopandas.read_file("../data/external/approx_ambulance_trust_shapes/approx_ambulance_trusts.shp")

fuel_pov_layer = geopandas.read_file("../data/external/layers.gpkg")
trusts_to_fuel = geopandas.sjoin(ambulance_trust, fuel_pov_layer, how="left").dropna(axis=0)
trusts_to_fuel.head()

,id_left,objectid_left,ccg21cd,ccg21nm,bng_e_left,bng_n_left,long_left,lat_left,shape__are_left,shape__len_left,...,lat_right,shape__are_right,shape__len_right,fuel poverty data_lsoa name,fuel poverty data_la code,fuel poverty data_la name,fuel poverty data_region,fuel poverty data_number of households1,fuel poverty data_number of households in fuel poverty1,fuel poverty data_proportion of households fuel poor (%)
0,133,27,E38000089,NHS Kernow CCG,212497,64493,-4.64254,50.45022,3.565119e+09,1.356815e+06,...,49.92333,1.638746e+07,103109.005881,Isles of Scilly 001A,E06000053,Isles of Scilly,South West,"1,107",130.0,12
0,133,27,E38000089,NHS Kernow CCG,212497,64493,-4.64254,50.45022,3.565119e+09,1.356815e+06,...,49.99874,2.930187e+07,31371.829019,Cornwall 073A,E06000052,Cornwall,South West,906,110.0,12
0,133,27,E38000089,NHS Kernow CCG,212497,64493,-4.64254,50.45022,3.565119e+09,1.356815e+06,...,50.01058,2.030131e+07,33109.696995,Cornwall 073D,E06000052,Cornwall,South West,"1,048",122.0,12
0,133,27,E38000089,NHS Kernow CCG,212497,64493,-4.64254,50.45022,3.565119e+09,1.356815e+06,...,50.04354,4.176496e+07,37734.630853,Cornwall 073E,E06000052,Cornwall,South West,976,128.0,13
0,133,27,E38000089,NHS Kernow CCG,212497,64493,-4.64254,50.45022,3.565119e+09,1.356815e+06,...,50.06494,3.125426e+07,33144.181452,Cornwall 073C,E06000052,Cornwall,South West,498,64.0,13


In [15]:
trusts_fuel = trusts_to_fuel[["ccg21nm","fuel poverty data_proportion of households fuel poor (%)"]].rename(columns={"fuel poverty data_proportion of households fuel poor (%)":"fuel_poverty"}).copy()
trusts_fuel["poverty_prop"] = pd.to_numeric(trusts_fuel["fuel_poverty"])
pov_per_trust = trusts_fuel[["ccg21nm","poverty_prop"]].groupby(["ccg21nm"]).mean().reset_index()
pov_per_trust.head()

,ccg21nm,poverty_prop
0,NHS Barnsley CCG,16.696206
1,NHS Basildon and Brentwood CCG,12.735452
2,NHS Blackburn with Darwen CCG,14.498308
3,NHS Cannock Chase CCG,17.396253
4,NHS Kernow CCG,10.552136


We then repeat the process, but this time using health deprivation (using the IMD index).

In [16]:
health_deprivation = geopandas.read_file("../data/external/hdd.gpkg")
health_deprivation.head()

,lsoa11cd,lsoa11nm,lsoa11nmw,st_areasha,st_lengths,imd_rank,imd_decile,lsoa01nm,ladcd,ladnm,...,inddec,outscore,outrank,outdec,totpop,depchi,pop16_59,pop60+,workpop,geometry
0,E01000001,City of London 001A,City of London 001A,133320.768872,2291.846072,29199,9,City of London 001A,E09000001,City of London,...,5,1.503,1615,1,1296,175,656,465,715.00,"MULTIPOLYGON (((532105.092 182011.230, 532162...."
1,E01000002,City of London 001B,City of London 001B,226191.272990,2433.960112,30379,10,City of London 001B,E09000001,City of London,...,7,1.196,2969,1,1156,182,580,394,619.75,"MULTIPOLYGON (((532746.813 181786.891, 532671...."
2,E01000003,City of London 001C,City of London 001C,57302.966538,1142.359799,14915,5,City of London 001C,E09000001,City of London,...,6,2.207,162,1,1350,146,759,445,804.00,"MULTIPOLYGON (((532135.145 182198.119, 532158...."
3,E01000005,City of London 001E,City of London 001E,190738.760504,2167.868343,8678,3,City of London 001E,E09000001,City of London,...,8,1.769,849,1,1121,229,692,200,683.00,"MULTIPOLYGON (((533807.946 180767.770, 533649...."
4,E01000006,Barking and Dagenham 016A,Barking and Dagenham 016A,144195.846857,1935.510354,14486,5,Barking and Dagenham 016A,E09000002,Barking and Dagenham,...,5,0.969,4368,2,2040,522,1297,221,1284.50,"MULTIPOLYGON (((545122.049 184314.931, 545271...."


In [17]:
trust_to_health_dep = geopandas.sjoin(ambulance_trust, health_deprivation, how="left").dropna(axis=0)


trust_health = trust_to_health_dep[["ccg21nm","hddscore"]]
trust_health["hddscore"] = pd.to_numeric(trust_health["hddscore"])

hdd_per_trust = trust_health[["ccg21nm","hddscore"]].groupby(["ccg21nm"]).mean().reset_index()
hdd_per_trust.head()

/tmp/ipykernel_32946/1512038373.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trust_health["hddscore"] = pd.to_numeric(trust_health["hddscore"])


,ccg21nm,hddscore
0,NHS Barnsley CCG,0.314100
1,NHS Basildon and Brentwood CCG,-0.342181
2,NHS Blackburn with Darwen CCG,0.621805
3,NHS Cannock Chase CCG,0.196028
4,NHS Kernow CCG,-0.166827


We then combined our two datasets together, and do some basic cleaning of trust names.

In [18]:
trust_with_all_metrics = ambulance_trust.merge(pov_per_trust, how="left", on="ccg21nm").merge(hdd_per_trust, how="left", on="ccg21nm")
trust_with_all_metrics

,id,objectid,ccg21cd,ccg21nm,bng_e,bng_n,long,lat,shape__are,shape__len,geometry,poverty_prop,hddscore
0,133,27,E38000089,NHS Kernow CCG,212497,64493,-4.642540,50.45022,3.565119e+09,1.356815e+06,"MULTIPOLYGON (((83998.680 5390.750, 83996.750 ...",10.552136,-0.166827
1,134,93,E38000244,NHS South East London CCG,541305,168583,0.029892,51.39868,3.488605e+08,1.248200e+05,"MULTIPOLYGON (((552838.400 175550.000, 552843....",15.106058,-0.397073
2,135,95,E38000246,NHS Surrey Heartlands CCG,516837,150388,-0.327610,51.24071,1.595693e+09,3.010981e+05,"MULTIPOLYGON (((503578.594 175452.610, 503574....",7.595187,-0.449243
3,136,40,E38000137,NHS Portsmouth CCG,465619,101352,-1.070060,50.80800,4.038664e+07,6.181838e+04,"MULTIPOLYGON (((465548.313 94917.149, 465547.5...",7.440331,-0.593025
4,137,2,E38000007,NHS Basildon and Brentwood CCG,564014,194421,0.368068,51.62470,2.631183e+08,1.337737e+05,"MULTIPOLYGON (((576397.460 184824.420, 576403....",12.735452,-0.342181
5,138,67,E38000212,NHS Newcastle Gateshead CCG,420165,559658,-1.686850,54.93120,2.557887e+08,1.547149e+05,"MULTIPOLYGON (((415220.399 565094.003, 415242....",14.724405,0.680426
6,139,4,E38000014,NHS Blackburn with Darwen CCG,369490,422806,-2.463600,53.70080,1.370126e+08,6.524908e+04,"MULTIPOLYGON (((347049.380 414681.250, 347043....",14.498308,0.621805
7,140,11,E38000028,NHS Cannock Chase CCG,401391,311481,-1.980840,52.70101,1.337217e+08,8.324228e+04,"POLYGON ((374544.500 344634.312, 374554.068 34...",17.396253,0.196028
8,141,92,E38000243,NHS Nottingham and Nottinghamshire CCG,466571,348422,-1.008830,53.02897,1.521517e+09,2.497150e+05,"MULTIPOLYGON (((447154.876 288685.246, 447137....",13.808985,0.064179
9,142,1,E38000006,NHS Barnsley CCG,429979,403330,-1.549250,53.52580,3.290521e+08,1.255186e+05,"MULTIPOLYGON (((413265.887 398534.505, 413261....",16.696206,0.314100


We then conduct some manual data cleaning to bring together trust names, as well as manually import the Cat2 mean call time, which we use as a metric of ambulance performance.

In [19]:
corrected_per = pd.read_csv("../data/interim/manual_corrections.csv").iloc[:,1:].copy()
corrected_per

,ccg21cd,ccg21nm,CorrectTrust,Time
0,E38000089,NHS Kernow CCG,South Western,9:50
1,E38000244,NHS South East London CCG,London,55:44
2,E38000246,NHS Surrey Heartlands CCG,South East Coast,35:31
3,E38000137,NHS Portsmouth CCG,South Central,43:28
4,E38000007,NHS Basildon and Brentwood CCG,East Midlands,11:51
5,E38000212,NHS Newcastle Gateshead CCG,North East,44:00
6,E38000014,NHS Blackburn with Darwen CCG,North West,39:46
7,E38000028,NHS Cannock Chase CCG,West Midlands,52:11
8,E38000243,NHS Nottingham and Nottinghamshire CCG,East of England,56:48
9,E38000006,NHS Barnsley CCG,Yorkshire,43:18


In [20]:
combined_gdf = trust_with_all_metrics.merge(corrected_per, how="left", on="ccg21cd")
combined_gdf

,id,objectid,ccg21cd,ccg21nm_x,bng_e,bng_n,long,lat,shape__are,shape__len,geometry,poverty_prop,hddscore,ccg21nm_y,CorrectTrust,Time
0,133,27,E38000089,NHS Kernow CCG,212497,64493,-4.642540,50.45022,3.565119e+09,1.356815e+06,"MULTIPOLYGON (((83998.680 5390.750, 83996.750 ...",10.552136,-0.166827,NHS Kernow CCG,South Western,9:50
1,134,93,E38000244,NHS South East London CCG,541305,168583,0.029892,51.39868,3.488605e+08,1.248200e+05,"MULTIPOLYGON (((552838.400 175550.000, 552843....",15.106058,-0.397073,NHS South East London CCG,London,55:44
2,135,95,E38000246,NHS Surrey Heartlands CCG,516837,150388,-0.327610,51.24071,1.595693e+09,3.010981e+05,"MULTIPOLYGON (((503578.594 175452.610, 503574....",7.595187,-0.449243,NHS Surrey Heartlands CCG,South East Coast,35:31
3,136,40,E38000137,NHS Portsmouth CCG,465619,101352,-1.070060,50.80800,4.038664e+07,6.181838e+04,"MULTIPOLYGON (((465548.313 94917.149, 465547.5...",7.440331,-0.593025,NHS Portsmouth CCG,South Central,43:28
4,137,2,E38000007,NHS Basildon and Brentwood CCG,564014,194421,0.368068,51.62470,2.631183e+08,1.337737e+05,"MULTIPOLYGON (((576397.460 184824.420, 576403....",12.735452,-0.342181,NHS Basildon and Brentwood CCG,East Midlands,11:51
5,138,67,E38000212,NHS Newcastle Gateshead CCG,420165,559658,-1.686850,54.93120,2.557887e+08,1.547149e+05,"MULTIPOLYGON (((415220.399 565094.003, 415242....",14.724405,0.680426,NHS Newcastle Gateshead CCG,North East,44:00
6,139,4,E38000014,NHS Blackburn with Darwen CCG,369490,422806,-2.463600,53.70080,1.370126e+08,6.524908e+04,"MULTIPOLYGON (((347049.380 414681.250, 347043....",14.498308,0.621805,NHS Blackburn with Darwen CCG,North West,39:46
7,140,11,E38000028,NHS Cannock Chase CCG,401391,311481,-1.980840,52.70101,1.337217e+08,8.324228e+04,"POLYGON ((374544.500 344634.312, 374554.068 34...",17.396253,0.196028,NHS Cannock Chase CCG,West Midlands,52:11
8,141,92,E38000243,NHS Nottingham and Nottinghamshire CCG,466571,348422,-1.008830,53.02897,1.521517e+09,2.497150e+05,"MULTIPOLYGON (((447154.876 288685.246, 447137....",13.808985,0.064179,NHS Nottingham and Nottinghamshire CCG,East of England,56:48
9,142,1,E38000006,NHS Barnsley CCG,429979,403330,-1.549250,53.52580,3.290521e+08,1.255186e+05,"MULTIPOLYGON (((413265.887 398534.505, 413261....",16.696206,0.314100,NHS Barnsley CCG,Yorkshire,43:18


In [21]:
combined_gdf[['Min', 'Sec']] = combined_gdf['Time'].str.split(':', 1, expand=True)
combined_gdf["Cat2Mins"] = combined_gdf["Min"].astype("int")
combined_gdf

,id,objectid,ccg21cd,ccg21nm_x,bng_e,bng_n,long,lat,shape__are,shape__len,geometry,poverty_prop,hddscore,ccg21nm_y,CorrectTrust,Time,Min,Sec,Cat2Mins
0,133,27,E38000089,NHS Kernow CCG,212497,64493,-4.642540,50.45022,3.565119e+09,1.356815e+06,"MULTIPOLYGON (((83998.680 5390.750, 83996.750 ...",10.552136,-0.166827,NHS Kernow CCG,South Western,9:50,9,50,9
1,134,93,E38000244,NHS South East London CCG,541305,168583,0.029892,51.39868,3.488605e+08,1.248200e+05,"MULTIPOLYGON (((552838.400 175550.000, 552843....",15.106058,-0.397073,NHS South East London CCG,London,55:44,55,44,55
2,135,95,E38000246,NHS Surrey Heartlands CCG,516837,150388,-0.327610,51.24071,1.595693e+09,3.010981e+05,"MULTIPOLYGON (((503578.594 175452.610, 503574....",7.595187,-0.449243,NHS Surrey Heartlands CCG,South East Coast,35:31,35,31,35
3,136,40,E38000137,NHS Portsmouth CCG,465619,101352,-1.070060,50.80800,4.038664e+07,6.181838e+04,"MULTIPOLYGON (((465548.313 94917.149, 465547.5...",7.440331,-0.593025,NHS Portsmouth CCG,South Central,43:28,43,28,43
4,137,2,E38000007,NHS Basildon and Brentwood CCG,564014,194421,0.368068,51.62470,2.631183e+08,1.337737e+05,"MULTIPOLYGON (((576397.460 184824.420, 576403....",12.735452,-0.342181,NHS Basildon and Brentwood CCG,East Midlands,11:51,11,51,11
5,138,67,E38000212,NHS Newcastle Gateshead CCG,420165,559658,-1.686850,54.93120,2.557887e+08,1.547149e+05,"MULTIPOLYGON (((415220.399 565094.003, 415242....",14.724405,0.680426,NHS Newcastle Gateshead CCG,North East,44:00,44,00,44
6,139,4,E38000014,NHS Blackburn with Darwen CCG,369490,422806,-2.463600,53.70080,1.370126e+08,6.524908e+04,"MULTIPOLYGON (((347049.380 414681.250, 347043....",14.498308,0.621805,NHS Blackburn with Darwen CCG,North West,39:46,39,46,39
7,140,11,E38000028,NHS Cannock Chase CCG,401391,311481,-1.980840,52.70101,1.337217e+08,8.324228e+04,"POLYGON ((374544.500 344634.312, 374554.068 34...",17.396253,0.196028,NHS Cannock Chase CCG,West Midlands,52:11,52,11,52
8,141,92,E38000243,NHS Nottingham and Nottinghamshire CCG,466571,348422,-1.008830,53.02897,1.521517e+09,2.497150e+05,"MULTIPOLYGON (((447154.876 288685.246, 447137....",13.808985,0.064179,NHS Nottingham and Nottinghamshire CCG,East of England,56:48,56,48,56
9,142,1,E38000006,NHS Barnsley CCG,429979,403330,-1.549250,53.52580,3.290521e+08,1.255186e+05,"MULTIPOLYGON (((413265.887 398534.505, 413261....",16.696206,0.314100,NHS Barnsley CCG,Yorkshire,43:18,43,18,43


We then add in the location of ambulance bases.

These were extracted using the OpenStreetMap Overpass API, using the query string
emergency=ambulance_station


In [22]:
ambulance_bases = geopandas.read_file("../data/external/export.geojson").to_crs("EPSG:27700")
ambulance_bases

,id,@id,CLA_PERS,CLA_PRES,COD_HAB,EMP_EST,ESTADO,FIXME,LOCALIDAD,NIVSOCIO,...,website,website_1,wheelchair,wheelchair:description,wheelchair:rental,wikidata,wikimedia_commons,wikipedia,zcen2011,geometry
0,node/31064825,node/31064825,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (2856221.144 -21588537.373)
1,node/33098255,node/33098255,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (1458738.095 -107141.368)
2,node/60016641,node/60016641,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (1057560.480 669198.060)
3,node/67972720,node/67972720,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (-5211521.847 3040770.444)
4,node/82599989,node/82599989,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (1423092.416 31563.693)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4969,node/9910441083,node/9910441083,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (1309609.777 -530707.386)
4970,node/9911136485,node/9911136485,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (7267816.678 -21434.370)
4971,node/9911196141,node/9911196141,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (4034703.086 -23049698.320)
4972,node/9912112027,node/9912112027,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (1148526.558 -645490.943)


Finally, we bring this together into our final map, display it and save as an output.

It's absolutely huge and unoptimised, so we don't actually visualise it here.

In [23]:

geoframe = combined_gdf
key = "CorrectTrust"
category = "poverty_prop"

values_to_show = ["hddscore"]

# Create interactive map with default basemap
map_osm = folium.Map(location=[51.5074, 0.1278], tiles='CartoDB positron')

heat = folium.Choropleth(
    geo_data=geoframe[[key, "geometry", category]].sort_values(by=category, ascending=False),
    name='Fuel Poverty',
    data=geoframe,
    columns=[key, category],
    fill_color="BuPu",
    show=False,
    key_on="feature.properties." + str(key)
)
map_osm.add_child(heat)

health_depriv = folium.Choropleth(
    geo_data=geoframe[[key, "geometry", "hddscore"]].sort_values(by="hddscore", ascending=False),
    name='Health Poverty',
    data=geoframe,
    columns=[key, "hddscore"],
    fill_color="BuPu",
    show=False,
    key_on="feature.properties." + str(key)
)
map_osm.add_child(health_depriv)

ambo_times = folium.Choropleth(
    geo_data=geoframe[[key, "geometry", "Cat2Mins"]].sort_values(by="Cat2Mins", ascending=False),
    name='Response Time',
    data=geoframe,
    columns=[key, "Cat2Mins"],
    fill_color="BuPu",
    key_on="feature.properties." + str(key)
)
map_osm.add_child(ambo_times)

ambo_bases = folium.GeoJson(ambulance_bases, name="Ambulance Bases", show=True)
map_osm.add_child(ambo_bases)

style_function = lambda x: {'fillColor': '#ffffff',
                            'color': '#000000',
                            'fillOpacity': 0.1,
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000',
                                'color': '#000000',
                                'fillOpacity': 0.50,
                                'weight': 0.1}
nil = folium.features.GeoJson(
    geoframe[[key, "geometry", category] + values_to_show].sort_values(by=category, ascending=False),
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=[key, category] + values_to_show,
        aliases=['Ref', 'Score'] + values_to_show,
        style="background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"
    )
)
map_osm.add_child(nil)
map_osm.keep_in_front(nil)
folium.LayerControl().add_to(map_osm)

for key in map_osm._children:
    if key.startswith('color_map'):
        del(map_osm[key])

#map_osm





In [24]:
map_osm.save("../outputs/ambo_map.html")